In [5]:
#DISPLAYING UNOPTIMIZED MAPS & Tables
#MAP
import requests
from pathlib import Path
from zipfile import ZipFile
import geopandas as gpd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from ipywidgets import Button, HBox, VBox,interact
from IPython.display import display, HTML
from IPython.display import clear_output
import ipywidgets as widgets
import numpy


# open it...

geodf = gpd.read_file("C:/Users/kyey/Documents/Region.geojson") ## Data with geometry
# shape file is a different CRS,  change to lon/lat GPS co-ordinates
geodf = geodf.to_crs("WGS84")

import pandas as pd
dat = pd.read_csv("C:/Users/kyey/Documents/DATA.csv",sep=";") #Simulated data

#Merging databases together - linking spatial information with simulated
geometry = pd.DataFrame(list(set(dat['id'])),geodf['geometry'])
d = {'geometry':geodf['geometry'], 'id1': list(set(dat['id']))}
geometry = pd.DataFrame(d)

geodf1 = dat.merge(geometry,left_on="id",right_on="id1")
geometry = geodf1['geometry']
geodf1.drop('geometry',axis=1)
geodf1 = gpd.GeoDataFrame(geodf1,crs="WGS84",geometry=geometry)


def selection_fn(trace,points,selector):
    if 't' in locals():
        t.data[0].cells.values = [geodf2.iloc[points.point_inds][col] for col in ['id','Harvested_V','SC','Biomass']]#variables of interest in map -- could be a cross box (?)
    global sel_stand
    bb = [[geodf2.iloc[points.point_inds][col] for col in ['id']]]
    sel_stand = [bb[0][0].iloc[i] for i in range(0,len(bb[0][0]))]
    
def on_display_map(**kwargs):
    display(kwargs)
    clear_output()
    geodf2 = geodf1[(geodf1['year'] == kwargs['year']) & (geodf1['regime'] == kwargs['regime'])]
    fig = px.choropleth_mapbox(geodf2.set_index("id"),    geojson=geodf2.geometry,    locations=geodf2.index,    color=kwargs['values'],    center=dict(lat= 62.82633, lon=21.259906),    mapbox_style="open-street-map",opacity =0.4,    zoom=13,)
    fig.update_layout(    height=500,    autosize=True,    margin={"r": 0, "t": 0, "l": 0, "b": 0})#
    ff =go.FigureWidget(fig)
    scatter = ff.data[0]
    scatter.on_selection(selection_fn)
    display(VBox([ff]))

def on_display_table(**kwargs):
    clear_output()
    if 'sel_stand' in globals():

        t = go.FigureWidget([go.Table(
            header=dict(values=['id']+[kwargs['1']+kwargs['2']+kwargs['3']],#variables of interest in map -- could be a cross box (?)
                        fill = dict(color='#C2D4FF'),
                        align = ['left'] * 5),
            cells=dict(values=[geodf2[col] for col in ['id']+[kwargs['1']+kwargs['2']+kwargs['3']]],#variables of interest in map -- could be a cross box (?)
                       fill = dict(color='#F5F8FF'),
                       align = ['left'] * 5))])
    display(VBox([t]))

def on_display_table(**kwargs):
    display(kwargs)
    clear_output()
    if 'sel_stand' in globals():
        if kwargs['Year of interest'] != "ALL" and kwargs['Regime of interest'] != "ALL":
            geodfv = geodf1[(geodf1['year'] == kwargs['Year of interest']) & (geodf1['regime'] == kwargs['Regime of interest'])]
        elif kwargs['Year of interest'] == "ALL":
            geodfv = geodf1[(geodf1['regime'] == kwargs['Regime of interest'])]
        elif kwargs['Regime of interest'] == "ALL":
            geodfv = geodf1[(geodf1['year'] == kwargs['Year of interest'])]
        else:
            geodfv = geodf1
        t = go.FigureWidget([go.Table(
            header=dict(values=['id']+[kwargs['Column 1'],kwargs['Column 2'],kwargs['Column 3']],#variables of interest in map -- could be a cross box (?)
                        fill = dict(color='#C2D4FF'),
                        align = ['left'] * 5),
            cells=dict(values=[geodf1[col] for col in ['id']+[kwargs['Column 1'],kwargs['Column 2'],kwargs['Column 3']]],#variables of interest in map -- could be a cross box (?)
                       fill = dict(color='#F5F8FF'),
                       align = ['left'] * 5))])
        display(VBox([t]))
    else:
        #display("Please display map first,selecting year and regime")
        t = go.FigureWidget([go.Table(
            header=dict(values=['id',"regime"]+[kwargs['Column 1'],kwargs['Column 2'],kwargs['Column 3']],#variables of interest in map -- could be a cross box (?)
                        fill = dict(color='#C2D4FF'),
                        align = ['left'] * 5),
            cells=dict(values=[geodf1[col] for col in ['id',"regime"]+[kwargs['Column 1'],kwargs['Column 2'],kwargs['Column 3']]],#variables of interest in map -- could be a cross box (?)
                       fill = dict(color='#F5F8FF'),
                       align = ['left'] * 5))])
        display(VBox([t]))
    

colTypeChooser = widgets.interactive(on_display_map,{"manual":True,"manual_name": "Display / Update Map"},**{'values':list(geodf1.columns)[1:],'year':list(set(geodf1.year)),'regime':list(set(geodf1.regime))})

TablecolTypeChooser = widgets.interactive(on_display_table,{"manual":True,"manual_name": "Display / Update Table"},**{'Year of interest':["ALL"]+list(set(geodf1.year)),'Regime of interest':["ALL"]+list(set(geodf1.regime)),'Column 1':list(geodf1.columns)[1:],'Column 2':list(geodf1.columns)[1:],'Column 3':list(geodf1.columns)[1:]})


def showGUI():
    display(HTML('''<style>
    .widget-label { min-width: 60% !important; }
    </style>'''))
    display(colTypeChooser)

    display(HTML('''<style>
    .widget-label { min-width: 60% !important; }
    </style>'''))
    display(TablecolTypeChooser)

In [6]:
showGUI()

interactive(children=(Dropdown(description='values', options=('year', 'regime', 'V', 'i_Vm3', 'Harvested_V', '…

interactive(children=(Dropdown(description='Year of interest', options=('ALL', 2051, 2056, 2061, 2066, 2071, 2…